## Exercício 2

#### Marcos Cesar Ribeiro de Camargo - 9278045
#### Rafael Augusto Monteiro - 9293095

###### Atenção: Esse código requer Python 3.6 por usar f-strings nos prints.


In [1]:
import numpy as np # Usado para aritmética de arrays da MLP
import pandas as pd # Usado para operações com os datasets

In [2]:
class MLP(object):
    model = None

    @staticmethod
    def f(net):
        return ( 1/ (1+ np.exp(-net)) )

    @staticmethod
    def df_dnet(f_net):
        return ( f_net * (1 - f_net) )

    def __init__(self, input_length=2, hidden_length=[3, 3], output_length=1, activation_function=f , d_activation_function=df_dnet):
        hidden_layer = list()
        previous_length = input_length
        for layer_length in hidden_length:
            hidden_layer.append(np.random.rand(layer_length, previous_length+1) - 0.5)
            previous_length = layer_length

        self.model = {
            'input_length': input_length, 
            'hidden_length': hidden_length, 
            'output_length': output_length, 
            'activation_function': activation_function.__func__, 
            'd_activation_function': d_activation_function.__func__,
            'hidden': hidden_layer,
            'output': (np.random.rand(output_length, previous_length+1) - 0.5),
        }

    # Função de classificação/regressão sobre um item
    def forward(self, x):
        # Recuperando valores do modelo
        hidden = self.model['hidden']
        hidden_length = len(self.model['hidden_length'])
        output = self.model['output']
        f = self.model['activation_function']
        df = self.model['d_activation_function']

        # Camadas Escondidas
        net_h = [None] * hidden_length
        f_net_h = [None] * hidden_length
        df_net_h = [None] * hidden_length
        
        
        # Adicionando 1 para multiplicar o Theta.
        previous_layer = np.pad(x, (0, 1), 'constant', constant_values=(1))
        for i in range(hidden_length):
            net_h[i] = np.sum(np.multiply(hidden[i], previous_layer), axis=1)
            f_net_h[i] = f(net_h[i])
            df_net_h[i] = df(f_net_h[i])
            previous_layer = np.pad(f_net_h[i], (0, 1), 'constant', constant_values=(1))

            
        # Camada de Saída
        net_o = np.sum(np.multiply(output, previous_layer), axis=1)
        f_net_o = f(net_o)
        df_net_o = df(f_net_o)

        # Retornando valores do forward.
        return {
            'net_h': net_h,
            'f_net_h': f_net_h,
            'df_net_h': df_net_h,
            'net_o': net_o,
            'f_net_o': f_net_o,
            'df_net_o': df_net_o,
        }

    # Função de treino da MLP
    def backpropagation(self, X, Y, eta=0.1, threshold=1e-3, alpha=0,max_inter = 200000):
        squaredError = 2*threshold
        hidden_length = len(self.model['hidden_length'])
        counter = 0
        
        dE2_dw_o_t = 0
        dE2_dw_h_t = [0] * hidden_length
        while(squaredError > threshold and counter < max_inter):
            squaredError = 0
            # Pra cada valor do conjunto de dados
            for x, y in zip(X, Y):
                # Calculando saída
                results = self.forward(x)
                #  Calculando o erro
                error = (y - results['f_net_o'])                
                squaredError += np.sum(np.power(error, 2))
                

                # Backwards camada de saída
                h = hidden_length - 1
                delta_o = error * results['df_net_o']
                f_net_h = np.pad(results['f_net_h'][h], (0, 1), 'constant', constant_values=(1))
                dE2_dw_o = np.multiply(np.array([-2*delta_o]).T, np.array([f_net_h]))
                
                # Backwards camada escondida
                delta_h = [None] * hidden_length
                dE2_dw_h = [None] * hidden_length
                delta = delta_o
                w_o_kj = self.model['output'][:,0:self.model['hidden_length'][h]] 

                # For reverso
                for i in reversed(range(1, hidden_length)):
                    # Cálculo das adaptações
                    delta_h[i] = np.array([results['df_net_h'][i]]) * np.dot(delta, w_o_kj)         
                    dE2_dw_h[i] = np.multiply(-2*delta_h[i].T,
                        np.pad(results['f_net_h'][i-1], (0, 1), 'constant', constant_values=(1)))
                    # Controlando as iterações
                    delta = delta_h[i]
                    w_o_kj = self.model['hidden'][i][:, 0:self.model['hidden_length'][i-1]]
                    
                delta_h[0] = np.array([results['df_net_h'][0]]) * np.dot(delta, w_o_kj)               
                dE2_dw_h[0] =  np.multiply(-2*delta_h[0].T,
                    np.pad(x, (0, 1), 'constant', constant_values=(1)))
                               
                
                # Aplicar adaptação na saída
                self.model['output'] = self.model['output'] - eta * dE2_dw_o - alpha*dE2_dw_o_t

                # Aplicar adaptação na escondida
                for i in range(hidden_length):
                    self.model['hidden'][i] = self.model['hidden'][i] - eta * dE2_dw_h[i] - alpha*dE2_dw_h_t[i]

                dE2_dw_o_t = dE2_dw_o
                dE2_dw_h_t = dE2_dw_h
                
            squaredError = squaredError / len(X) 
            counter += 1
            if(counter % 100 == 0):
                print('currently with error %.6lf - on iter. %d' % (squaredError, counter))
        print(f'error {squaredError:.6f} - iter. {counter}')

## Testes

As funções abaixo servem para operações auxiliares sobre os datasets e sobre os resultados da MLP.

In [66]:
######## Funções Auxiliares #########

# Função de normalização chamada para cada coluna
def norm_f(col):
    return (col - col.min()) / (col.max() - col.min())

# Função para normalizar um dataframe no intervalo [0,1]
def normalize_df(df):
    return df.apply(norm_f, axis=0)

# Função para realizar o split do dataset em treino e teste
def split_dataset(df, split_percentage):
    train_size = int(df.shape[0] * 0.8) # dataset de treino: 80% do total
    train = df.iloc[:train_size,:]
    test = df.iloc[train_size:,:]
    return (train, test)

# Função para classificar um conjunto de dados inteiro. 
def batch_classify(classifier, data, discretize = True):
    if discretize: # se eu precisar trocar os valores reais por binarios (caso wine)
        result = []
        for instance in data:
            predict = classifier.forward(instance)['f_net_o']
            result.append(np.where(predict == predict.max(),1,0))
        return np.array(result)
    else: # se eu nao precisar trocar nada, posso soh usar list comprehension
        return np.array([classifier.forward(instance)['f_net_o'] for instance in data])

# Função para transformar uma lista de listas de variáveis dummies em uma lista de classes
def undummy(y):
    result = []
    for item in y:
        if item[0] == 1:
            result.append(1)
        elif item[1] == 1:
            result.append(2)
        elif item[2] == 1:
            result.append(3)
    return result

# Função para calcular acurácia média dado uma lista de predições e uma lista de classes reais
def accuracy(y_true, y_pred):
    score = 0;
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            score += 1
    return score/len(y_true)

# Função para calcular erro médio quadrático
def mse(y_true, y_pred):
    return (np.sum((y_pred - y_true) ** 2)) / (y_pred.shape[0] * y_pred.shape[1])

# Função para printar o resultado dos testes de maneira mais bonita
def print_results(result):  
    print(f'Test dataset Accuracy: {result["test"]:.0%}  //  Train dataset Accuracy: {result["train"]:.0%}')
    
# Função para printar o resultado dos MSE de maneira mais bonita
def print_results_mse(result):  
    print(f'Test dataset Mean Squared Error: {result["test"]:.3f}  //  Train dataset Mean Squared Error: {result["train"]:.3f}')

### Dataset: Wine

A função abaixo realiza todos os passos para a avaliação da MLP sobre o dataset wine, com excessão da leitura do arquivo. Primeiro, é feita a substituição da classe (1, 2 ou 3) por variáveis dummies (são criados atributos 1, 2 e 3, e cada instância possui 0 ou 1 em cada atributo conforme a sua classe). 

Em seguida, as instâncias do dataset são aleatorizadas. 

No próximo passo, todos os campos do dataset são normalizados em 0-1. 

Então, o dataset é dividido em treino e teste, de acordo com o parâmetro *train_percentage*. 

Em seguida, as classes e os dados são separados. 

Finalmente, é realizado o treino da MLP, com os parâmetros:
* *layers*: formato das camadas ocultas. Uma camada com 10 neurônios por default
* *max_cycles*: número máximo de iterações do treinamento. 20k por default
* *momentum*: valor do foward momentum. 0.3 por default
* *eta*: taxa de aprendizado. 0.1 por default

Em seguida, é realizado o *batch_classify*, classificando todas as instâncias dos conjuntos de treino e teste. Tais classificações passam pelo *undummy* (As classes são convertidas novamente a 1,2 ou 3). 

Por fim, é feito o cálculo da acurácia das classificações no conjunto de treinamento e teste, e o resultado é retornado.

In [11]:
# Função que realiza todos os procedimentos para o teste da MLP conforme as especificações do trabalho
def classification_test_wine(dataset, layers = [10], max_cycles = 20000, momentum = 0.3, eta = 0.1, train_percentage = 0.8):

    # Substituindo classes por variáveis dummies
    df = pd.concat([pd.get_dummies(dataset['Class']), dataset.iloc[:,1:]], axis=1)
    
    # Aleatorizando as entradas
    df = df.sample(frac=1)
    
    # Normalizando as colunas para o intervalo [0, 1]
    df = normalize_df(df)
    
    # Dividindo o dataset dataset
    train, test = split_dataset(df, train_percentage)
    
    # Separando classes dos datasets
    train_target = train.iloc[:,:3].values
    train_data = train.drop(columns=[1,2,3]).values
    test_target = test.iloc[:,:3].values
    test_data = test.drop(columns=[1,2,3]).values
    
    # Treinando a mlp
    mlp = MLP(input_length = train_data.shape[1], hidden_length=layers, output_length = train_target.shape[1])
    mlp.backpropagation(train_data, train_target, alpha=momentum, threshold=1e-3, max_inter=max_cycles)
    
    # Realizando os testes
    test_predict = undummy(batch_classify(mlp, test_data))
    test_target = undummy(test_target)
    train_predict = undummy(batch_classify(mlp, train_data))
    train_target = undummy(train_target)

    return {'test': accuracy(test_target, test_predict), 'train': accuracy(train_target, train_predict)}

### Exemplos

No código abaixo, o dataset Wine é carregado e alguns testes são realizados com diversos parâmetros diferentes. Para cada execução, são mostrados o número de iterações do treinamento, o erro do treinamento, e a acurácia nos conjuntos de teste e treinamento

In [ ]:
# Carregando o dataset
wine = pd.read_csv('wine.csv', header=None ,names=['Class','Alcohol','Acid','Ash','Alca','Magnesium','TotalPhe','Flavanoids','Nonflavanoid','Proantho','CIntensity','Hue','Diluted','Proline'])

for max_iter in range(50,201,50): #checks from 50 to 200
    for momentum in [x * .1 for x in range(1, 5, 1)]: # checks from .1 to .4
        for eta in [x * .1 for x in range(1, 5, 1)]: # checks from .1 to .4
            for train_percentage in [x * .1 for x in range(5, 10, 1)]: # checks from .5 to .9
                for l in range(1,3,1): # checks from 1 layer to 2 layers
                    print(f'layers: {l}, train %: {train_percentage:.0%}, eta: {eta: .1f}, momentum: {momentum: .1f},  max_iter: {max_iter}')
                    result = classification_test_wine(wine, max_cycles = max_iter, momentum = momentum, eta = eta, train_percentage = train_percentage)
                    print_results(result)
                    print()

layers: 1, train %: 50%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0.011454 - iter. 50
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 2, train %: 50%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0.016734 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 60%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0.009860 - iter. 50
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 2, train %: 60%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0.017857 - iter. 50
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 1, train %: 70%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0.018426 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 70%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0.018231 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 80%, eta:  0.1, momentum:  0.1,  max_iter: 50
error 0

error 0.008751 - iter. 50
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 1, train %: 70%, eta:  0.2, momentum:  0.2,  max_iter: 50
error 0.013215 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 70%, eta:  0.2, momentum:  0.2,  max_iter: 50
error 0.006164 - iter. 50
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 1, train %: 80%, eta:  0.2, momentum:  0.2,  max_iter: 50
error 0.009091 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 80%, eta:  0.2, momentum:  0.2,  max_iter: 50
error 0.010986 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 90%, eta:  0.2, momentum:  0.2,  max_iter: 50
error 0.015189 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 90%, eta:  0.2, momentum:  0.2,  max_iter: 50
error 0.005334 - iter. 50
Test dataset Accuracy: 94%  //  Train dataset 

error 0.003829 - iter. 50
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 1, train %: 90%, eta:  0.3, momentum:  0.3,  max_iter: 50
error 0.010938 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 90%, eta:  0.3, momentum:  0.3,  max_iter: 50
error 0.008553 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 50%, eta:  0.4, momentum:  0.3,  max_iter: 50
error 0.005105 - iter. 50
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 2, train %: 50%, eta:  0.4, momentum:  0.3,  max_iter: 50
error 0.012328 - iter. 50
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 60%, eta:  0.4, momentum:  0.3,  max_iter: 50
error 0.004499 - iter. 50
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 2, train %: 60%, eta:  0.4, momentum:  0.3,  max_iter: 50
error 0.007711 - iter. 50
Test dataset Accuracy: 97%  //  Train dataset A

error 0.008328 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 60%, eta:  0.1, momentum:  0.1,  max_iter: 100
error 0.010087 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 60%, eta:  0.1, momentum:  0.1,  max_iter: 100
error 0.004775 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 70%, eta:  0.1, momentum:  0.1,  max_iter: 100
error 0.004389 - iter. 100
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 2, train %: 70%, eta:  0.1, momentum:  0.1,  max_iter: 100
error 0.006437 - iter. 100
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 1, train %: 80%, eta:  0.1, momentum:  0.1,  max_iter: 100
error 0.007615 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 80%, eta:  0.1, momentum:  0.1,  max_iter: 100
error 0.005272 - iter. 100
Test dataset Accuracy: 97%  //  T

error 0.004276 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 70%, eta:  0.2, momentum:  0.2,  max_iter: 100
error 0.004691 - iter. 100
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 1, train %: 80%, eta:  0.2, momentum:  0.2,  max_iter: 100
error 0.002193 - iter. 100
Test dataset Accuracy: 92%  //  Train dataset Accuracy: 100%

layers: 2, train %: 80%, eta:  0.2, momentum:  0.2,  max_iter: 100
error 0.004174 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 90%, eta:  0.2, momentum:  0.2,  max_iter: 100
error 0.003753 - iter. 100
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 2, train %: 90%, eta:  0.2, momentum:  0.2,  max_iter: 100
error 0.003921 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 50%, eta:  0.3, momentum:  0.2,  max_iter: 100
error 0.002749 - iter. 100
Test dataset Accuracy: 97%  //  Tr

error 0.001674 - iter. 100
Test dataset Accuracy: 97%  //  Train dataset Accuracy: 100%

layers: 1, train %: 90%, eta:  0.3, momentum:  0.3,  max_iter: 100
error 0.002266 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 2, train %: 90%, eta:  0.3, momentum:  0.3,  max_iter: 100
error 0.001998 - iter. 100
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 1, train %: 50%, eta:  0.4, momentum:  0.3,  max_iter: 100
error 0.001509 - iter. 100
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 2, train %: 50%, eta:  0.4, momentum:  0.3,  max_iter: 100
error 0.002270 - iter. 100
Test dataset Accuracy: 100%  //  Train dataset Accuracy: 100%

layers: 1, train %: 60%, eta:  0.4, momentum:  0.3,  max_iter: 100
error 0.002005 - iter. 100
Test dataset Accuracy: 94%  //  Train dataset Accuracy: 100%

layers: 2, train %: 60%, eta:  0.4, momentum:  0.3,  max_iter: 100
error 0.002222 - iter. 100
Test dataset Accuracy: 100%  //  Tr

### Dataset: default_features_1059_tracks

De forma similar, a função abaixo realiza todos os passos para a avaliação do dataset default_features_1059_tracks. As diferenças da função do dataset Wine é que não é necessário o uso de variáveis dummy, e o resultado calculado é o erro quadrático médio.

In [70]:
# Função que realiza todos os procedimentos para o teste da MLP conforme as especificações do trabalho
def classification_test_tracks(dataset, layers = [10], max_cycles = 500, momentum = 0.3, eta = 0.1, train_percentage = 0.8,  threshold=1e-3):
    
    # Aleatorizando as entradas
    df = dataset.sample(frac=1)
    
    # Normalizando as colunas para o intervalo [0, 1]
    df = normalize_df(df)
    
    # Dividindo o dataset dataset
    train, test = split_dataset(df, train_percentage)
    
    # Separando classes dos datasets
    train_target = train.iloc[:,68:].values # pega colunas 69 e 70
    train_data = train.drop(columns=[69,70]).values # pega colunas de 1 a 68
    test_target = test.iloc[:,68:].values
    test_data = test.drop(columns=[69,70]).values
    
    # Treinando a mlp
    mlp = MLP(input_length = train_data.shape[1], hidden_length=layers, output_length = train_target.shape[1])
    print('treinando mlp')
    mlp.backpropagation(train_data, train_target, alpha=momentum, threshold=threshold, max_inter=max_cycles)
    
    # Realizando os testes
    test_predict = batch_classify(mlp, test_data, discretize = False)
    test_target = test_target
    train_predict = batch_classify(mlp, train_data, discretize = False)
    train_target = train_target
    
    return {'test': mse(test_target, test_predict), 'train': mse(train_target, train_predict)}

### Exemplos

No código abaixo, o dataset default_features_1059_tracks é carregado e alguns testes são realizados com diversos parâmetros diferentes. Para cada execução, são mostrados o número de iterações do treinamento, o erro do treinamento, e os erros médios quadráticos para os conjuntos de teste e treinamento

In [71]:
# Carregando o dataset
tracks = pd.read_csv('default_features_1059_tracks.csv', header=None ,names=range(1,71))

result = classification_test_tracks(tracks, threshold=3*1e-2, eta=0.3, max_cycles=50)
print_results_mse(result)

treinando mlp
error 0.059120 - iter. 50
Test dataset Mean Squared Error: 0.033  //  Train dataset Mean Squared Error: 0.029
